In [1]:
from __future__ import annotations

import json
import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RefMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
async def dummy_loader(name: str) -> dict:
    await sleep(0.2)
    return {
        'name': name,
        'age': 20
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RefMap(
            ref=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RefMap(
                    ref=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RefMap(
                            ref=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RefMap(
                    ref=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    rel=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [7]:
with cProfile.Profile() as pr:
  db = DataBase()
  rec = await data_source.load([data], db=db)
  
  stats = pstats.Stats(pr)

Async-loading `Search`:   0%|          | 0/1 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 with cProfile.Profile() as pr:                                                               │
│   2   db = DataBase()                                                                            │
│ ❱ 3   rec = await data_source.load([data], db=db)                                                │
│   4                                                                                              │
│   5   stats = pstats.Stats(pr)                                                                   │
│   6                                                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:818 in load                   │
│                                                                                                  │
│   815 │   │   db = db if db is not None else DataBase()                                          │
│   816 │   │   in_data: InData = {((), ()): dat for dat in data}                                  │
│   817 │   │   rest_data: RestData = []                                                           │
│ ❱ 818 │   │   loaded = await _load_records(                                                      │
│   819 │   │   │   db, cast(RecMap[Record, TreeNode], self), in_data, rest_data                   │
│   820 │   │   )                                                                                  │
│   821                                                                                            │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:715 in _load_records          │
│                                                                                                  │
│   712 │   records: dict[Hashable, Record | Hashable] = {}                                        │
│   713 │   rest_records: InData = {}                                                              │
│   714 │                                                                                          │
│ ❱ 715 │   async for ((parent_idx, path_idx), data), (rec, rel) in loaded:                        │
│   716 │   │   if rec is None:                                                                    │
│   717 │   │   │   rest_records[(parent_idx, path_idx)] = data                                    │
│   718 │   │   │   continue                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: cannot unpack non-iterable AssertionError object

In [ ]:
[s for s in db[Search]]

In [ ]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")